In [ ]:
import os
import sys
import yaml

sys.path.append('./')
sys.path.append('../')

from src.predictions.rudderstack_predictions.connectors.SnowflakeConnector import SnowflakeConnector

In [ ]:
homedir = os.path.expanduser("~")

with open(os.path.join(homedir, ".pb/siteconfig.yaml"), "r") as f:
    creds = yaml.safe_load(f)["connections"]["shopify_wh"]["outputs"]["dev"]

creds["schema"] = "PROFILES_INTEGRATION_TEST"
if creds["type"] == "snowflake":
    print(f"Using {creds['schema']} schema in snowflake account: {creds['account']}")

## Define variables

In [ ]:
thresold_date = '2024-03-10'
creds["schema"] = "NAME_OF_THE_SCHEMA"

In [ ]:
connector = SnowflakeConnector()
session = connector.build_session(creds)

In [ ]:
get_stage_name_query = f"""SELECT STAGE_NAME
                                FROM INFORMATION_SCHEMA.STAGES
                                WHERE STAGE_CATALOG = '{creds['dbname']}'
                                AND STAGE_SCHEMA = '{creds['schema']}'
                                AND DATE(CREATED) < '{thresold_date}';"""

In [ ]:
stage_names = connector.run_query(session, get_stage_name_query)

In [ ]:
num_rows = len(stage_names)
temp = 1

for row in stage_names:
    delete_stage_query = f"""drop stage if exists {creds['dbname']}.{creds['schema']}.{row.STAGE_NAME};"""
    connector.run_query(session, delete_stage_query)
    if temp%50==0:
        print(f"{temp} out of {num_rows} stages deleted")
    temp+=1